# Now that all necessary data have now been extracted, transformed, and dumped to parquet files we can start doing our analyses by reading these parquet files as tables with duckdb (an in process OLAP) and then use these tables to make our transformations and draw insights

In [1]:
import duckdb
import os
from dotenv import load_dotenv
from pathlib import Path

%load_ext autoreload
%autoreload 2

In [ ]:
# C:\Users\LARRY\Documents\Scripts\data-engineering-path\chronic-disease-analyses\chronic_disease_analyses_db.db
conn = duckdb.connect("chronic_disease_analyses_db.db")

In [3]:
tables = [
    table
    for table_tuple in conn.sql("""SHOW TABLES""").fetchall()
    for table in table_tuple 
]
tables

['CDI',
 'CDILocation',
 'CDIStratification',
 'DataValueType',
 'Population',
 'PopulationState',
 'PopulationStratification',
 'Question']

In [4]:
for table in tables:
    counts = conn.sql(f"""SELECT COUNT(*) FROM {table}""").fetchall()[0][0]
    print(counts)

678471
51
11
15
2526336
51
24
192


In [5]:
conn.sql("""
    SELECT * FROM CDILocation
""")

┌────────────┬────────────────┬─────────────────────┬────────────────────┐
│ LocationID │  LocationDesc  │      Latitude       │     Longitude      │
│  varchar   │    varchar     │       double        │       double       │
├────────────┼────────────────┼─────────────────────┼────────────────────┤
│ NM         │ New Mexico     │ -106.24058098499967 │  34.52088095200048 │
│ OK         │ Oklahoma       │  -97.52107021399968 │  35.47203135600046 │
│ GA         │ Georgia        │  -83.62758034599966 │  32.83968109300048 │
│ KY         │ Kentucky       │  -84.77497104799966 │ 37.645970271000465 │
│ ND         │ North Dakota   │ -100.11842104899966 │  47.47531977900047 │
│ WY         │ Wyoming        │ -108.10983035299967 │  43.23554134300048 │
│ TN         │ Tennessee      │  -85.77449091399967 │  35.68094058000048 │
│ LA         │ Louisiana      │  -92.44568007099969 │  31.31266064400046 │
│ NE         │ Nebraska       │  -99.36572062299967 │   41.6410409880005 │
│ AK         │ Alaska    

In [6]:
conn.sql("""
    SELECT * FROM CDIStratification
""")

┌─────────┬─────────────┬──────────────┬──────────────────┐
│   Sex   │  Ethnicity  │    Origin    │ StratificationID │
│ varchar │   varchar   │   varchar    │     varchar      │
├─────────┼─────────────┼──────────────┼──────────────────┤
│ Male    │ All         │ Both         │ B_M_ALL          │
│ Both    │ White       │ Not Hispanic │ NH_B_WHITE       │
│ Both    │ Black       │ Not Hispanic │ NH_B_BLACK       │
│ Both    │ AIAN        │ Not Hispanic │ NH_B_AIAN        │
│ Both    │ Multiracial │ Not Hispanic │ NH_B_MULTI       │
│ Both    │ NHPI        │ Not Hispanic │ NH_B_NHPI        │
│ Both    │ Other       │ Not Hispanic │ NH_B_OTHER       │
│ Female  │ All         │ Both         │ B_F_ALL          │
│ Both    │ All         │ Hispanic     │ H_B_ALL          │
│ Both    │ All         │ Both         │ B_B_ALL          │
│ Both    │ Asian       │ Not Hispanic │ NH_B_ASIAN       │
├─────────┴─────────────┴──────────────┴──────────────────┤
│ 11 rows                               

In [7]:
conn.sql("""
    SELECT * FROM DataValueType
""")

┌───────────────────┬──────────────────────────────────────────┐
│  DataValueTypeID  │              DataValueType               │
│      varchar      │                 varchar                  │
├───────────────────┼──────────────────────────────────────────┤
│ MEDIAN            │ Median                                   │
│ NMBR              │ Number                                   │
│ AGEADJRATE        │ Age-adjusted Rate                        │
│ PERCAPALC         │ Per capita alcohol consumption           │
│ USD               │ US Dollars                               │
│ AGEADJMEAN        │ Age-adjusted Mean                        │
│ PREV              │ Prevalence                               │
│ MEAN              │ Mean                                     │
│ AVGANNCRDRATE     │ Average Annual Crude Rate                │
│ CRDRATE           │ Crude Rate                               │
│ AGEADJPREV        │ Age-adjusted Prevalence                  │
│ AVGANNAGEADJRATE  │ Ave

In [8]:
conn.sql("""
    SELECT * FROM Question
""")

┌────────────┬─────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────┬──────────┬────────┐
│ QuestionID │ TopicID │                                                   Question                                                   │                      Topic                      │ AgeStart │ AgeEnd │
│  varchar   │ varchar │                                                   varchar                                                    │                     varchar                     │  double  │ double │
├────────────┼─────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────┼─────────────────────────────────────────────────┼──────────┼────────┤
│ ART1_1     │ ART     │ Arthritis among adults aged >= 18 years                                                                      │ Arthritis                               

In [9]:
conn.sql("""
    SELECT * FROM Population
""")

┌─────────┬───────┬──────────────────┬───────┬────────────┐
│ StateID │  Age  │ StratificationID │ Year  │ Population │
│ varchar │ float │     varchar      │ int32 │   int64    │
├─────────┼───────┼──────────────────┼───────┼────────────┤
│ AL      │   0.0 │ NH_M_WHITE       │  2000 │      19270 │
│ AL      │   0.0 │ NH_M_WHITE       │  2001 │      19612 │
│ AL      │   0.0 │ NH_M_WHITE       │  2002 │      18731 │
│ AL      │   0.0 │ NH_M_WHITE       │  2003 │      18623 │
│ AL      │   0.0 │ NH_M_WHITE       │  2004 │      18659 │
│ AL      │   0.0 │ NH_M_WHITE       │  2005 │      18816 │
│ AL      │   0.0 │ NH_M_WHITE       │  2006 │      18877 │
│ AL      │   0.0 │ NH_M_WHITE       │  2007 │      19027 │
│ AL      │   0.0 │ NH_M_WHITE       │  2008 │      18937 │
│ AL      │   0.0 │ NH_M_WHITE       │  2009 │      18039 │
│ ·       │    ·  │     ·            │    ·  │          · │
│ ·       │    ·  │     ·            │    ·  │          · │
│ ·       │    ·  │     ·            │  

In [10]:
conn.sql("""
    SELECT * FROM PopulationState
""")

┌───────────────┬─────────┐
│     State     │ StateID │
│    varchar    │ varchar │
├───────────────┼─────────┤
│ Mississippi   │ MS      │
│ South Dakota  │ SD      │
│ Utah          │ UT      │
│ Kentucky      │ KY      │
│ California    │ CA      │
│ Nebraska      │ NE      │
│ New Hampshire │ NH      │
│ Delaware      │ DE      │
│ Minnesota     │ MN      │
│ Nevada        │ NV      │
│   ·           │ ·       │
│   ·           │ ·       │
│   ·           │ ·       │
│ Michigan      │ MI      │
│ Illinois      │ IL      │
│ Maryland      │ MD      │
│ Alaska        │ AK      │
│ Connecticut   │ CT      │
│ Vermont       │ VT      │
│ Massachusetts │ MA      │
│ Oklahoma      │ OK      │
│ West Virginia │ WV      │
│ Virginia      │ VA      │
├───────────────┴─────────┤
│   51 rows (20 shown)    │
└─────────────────────────┘

In [11]:
conn.sql("""
    SELECT * FROM PopulationStratification
""")

┌─────────┬─────────────┬──────────────┬──────────────────┐
│   Sex   │  Ethnicity  │    Origin    │ StratificationID │
│ varchar │   varchar   │   varchar    │     varchar      │
├─────────┼─────────────┼──────────────┼──────────────────┤
│ Female  │ Black       │ Not Hispanic │ NH_F_BLACK       │
│ Female  │ Asian       │ Not Hispanic │ NH_F_ASIAN       │
│ Female  │ NHPI        │ Hispanic     │ H_F_NHPI         │
│ Male    │ Asian       │ Not Hispanic │ NH_M_ASIAN       │
│ Male    │ White       │ Not Hispanic │ NH_M_WHITE       │
│ Male    │ AIAN        │ Not Hispanic │ NH_M_AIAN        │
│ Male    │ Multiracial │ Hispanic     │ H_M_MULTI        │
│ Male    │ Black       │ Not Hispanic │ NH_M_BLACK       │
│ Male    │ Black       │ Hispanic     │ H_M_BLACK        │
│ Female  │ Asian       │ Hispanic     │ H_F_ASIAN        │
│  ·      │   ·         │    ·         │     ·            │
│  ·      │   ·         │    ·         │     ·            │
│  ·      │   ·         │    ·         │

In [12]:
# # Loading CDI dimension tables
# #### location table
# cdi_location_url = "s3://chronic-disease-analyses-bucket/cdi-data-transformed/Location.parquet/*.parquet"
# query = f"""
#     SELECT *
#     FROM read_parquet('{cdi_location_url}', union_by_name=True, filename=False)
# """
# conn.sql(query)

In [13]:
conn.close()